# Homework 4 - Code Search using Deep Learning Bi-Encoders

Modern search engines are capable of working with more than just text: images, videos, audio, songs, ... the list goes on and on! In Homework 4, you'll develop an IR system that works in two modalities: text queries and _code_ documents. Code may be written in text characters (it's human readable, unlike image files) but queries like ``python function to test if file exists`` may look nothing like the code that matches the query's intent.

To bridge these two languages, we'll use a relative recent technique known as a *bi-encoder* model to convert text queries and code documents into the same representation space. If you recall from class, our tf-idf vectorizer was an encoder—-it would convert some text query into a high-dimensional vector, _encoding_ the text into some numeric space. Here, we'll use deep learning encoders that also produce numeric vectors. However, unlike our tf-idf vectors which are large (the size of the vocabulary) and sparse (most queries don't have most words), our deep learning-produced vectors will be relatively small (hundreds of dimensions) and dense (all dimensions have a non-zero value). 

For text, most of the common deep learning models are similar to the **BERT** model we talked about in class during the deep learning week. If you recall, this was the model that was trained using a MadLibs-style approach where it's shown most of a sentence and asked to fill in the blanks for a few words that have been removed. This type of training lets the model ultimately produce a dense vector representation for the "meaning" of a text input--essentially acting as a text encoder. If you're curious, there are a few [good](https://huggingface.co/blog/bert-101) [tutorials](https://wandb.ai/mukilan/BERT_Sentiment_Analysis/reports/An-Introduction-to-BERT-And-How-To-Use-It--VmlldzoyNTIyOTA1) on what this model is all about, though you don't need to know it for the purposes of this assignment. The most relevant thing to know is that other folks have adapted this approach to produce other models that can encode the meanings of other kinds of input, _like code_.

<img src="https://weaviate.io/img/blog/cross-encoders/bi-encoder.png" alt="drawing" width="200"/>

The bi-encoder architecture (shown above for two _sentence_ inputs) use two of these deep learning encoder models to compare the encodings of each model's input in some way. Here, we'll compare an encoded query and an encoded-code-document to measure the IR _relevance_ of that document for the given query. The bi-encoder setup lets us bridge between the two languages by _training the model to adjust its representations_. The key idea here is that both encoders know how to represent each of their respective inputs (they've already been pre-trained to do this) and by further training, we can get the encoders to adjust how they represent things so that representations for very relevant query-document papers look similar and representations for non-relevant pairs look dissimilar.

### Homework summary

This homework provides a substantial amount of code to get you started and has you focus on just a few implementation pieces.

* **Part 1** of the homework will be training the bi-encoder model. You will need a GPU for this, so you'll want to run this on a Great Lakes machine with a GPU. We've done testing and set the hyperparameters below so that it should run quickly enough. _But_ it will still take an hour to complete training once you get it up and running. Once you have the model trained, you'll export the query-document relevance scores from the bi-encoder to a file for use later.

* **Part 2** will have you learning how to do Learning to Rank (L2R) again and incorporating the bi-encoder's predictions. For this part, you will need to have access to Java to run Pyterrier. However, you will not need a GPU and this part can be run either on a non-GPU Great Lakes machine or on your local machine. You will still need access to the outputs of the bi-encoder which are in a file, so if you work locally, you'll need to copy those still. Part 2 will show you how to set up new pipelines that make use of precomputed features.

For both parts, we've put in extensive comments throughout so you can get a sense of what is happening and why. You don't need to understand all of the code, but it will be helpful to try to understand the general flow of things and how you train these kinds of models at a high level (possibly relating to questions on the final exam).

### Implementation Notes 

This notebook contains a lot of code that is intended to expose you to various deep learning concepts _but_ where you don't need to modify it. To streamline the process, we've put even more low-level code into two files that you don't need to read: 

- **edited_roberta.py** contains the pre-trained RoBERTa model architecture.
- **run_classifiers.py** contains the some helpful scripts for running things and loading data.

If you've never seen PyTorch code, that's ok (and expected!). We've added detailed instructions on what you need to know or learn in each step. Your pytorch code will be very (_very_) minimal, but we do want you to try a few things. There are also many good tutorials out there for pytorch _but_ these tutorial cover more information than what is needed too, so please read judiciously so you don't feel overwhelmed. If you do decide to read more, perhaps one general place to start is tutorials on using PyTorch for [logistic regression](https://towardsdatascience.com/logistic-regression-with-pytorch-3c8bbea594be).


### What to do

For Part 1, we've marked your parts with TODO. To get a sense of what and why you're doing something, be sure to read the nearby parts of the notebook for context. 

In general, you will need a GPU to run this. However, we've provided a very tiny input file `data/train_10.txt` that has only 10 training examples. Training only on this file will produce horrible results. _However_ it is small enough that you can probably run the code on a CPU to start debugging, though this may take a lot of memory. Worst case, you can start and debug this file on a Great Lakes machine (without a GPU) and once you think it's working, switch to a GPU.


### Learning goals for Part 1 of Homework 4:
* Expose you to how to run and train deep learning systems
* Gain familiarity with bi-encoder models for IR with different modalities
* Learn how to read and import data for training bi-encoder models
* Learn how to export relevance predictions from deep learning models for use in IR ranking systems


In [1]:
!pip install tensorboardX
!pip install python-terrier
!pip install transformers


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
from edited_roberta import *
from run_classifier import evaluate, load_and_cache_examples, accuracy, set_seed
import argparse
import glob
import logging
import os
import random
import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm, trange
from transformers import (WEIGHTS_NAME, get_linear_schedule_with_warmup, AdamW,RobertaConfig,RobertaTokenizer)
from utils import (compute_metrics, convert_examples_to_features,
                        output_modes, processors)

import pandas as pd

from torch.nn.functional import cosine_similarity

logger = logging.getLogger(__name__)

# Defining our Bi-Encoder model

In the following block, we'll create our neural network bi-encoder model. To do so, we'll define a new class `CodeSearchBiencoderModel` that contains the two encoders, which are the encoder for query and the encoder for code. 

In pytorch, neural networks are defined by specifying their _parameters_ (the things that get updated during training) and a `forward` function that determines how to turn the inputs into outputs. For our bi-encoder, we'll need to fill these in as follows:

* Specify the two encoders in the `__init__` function as fields of the class (e.g., `self.x = 1` makes `x` a field of the object), which will tell PyTorch that we'll be updating their parameters during training
* Write the `forward` function so that we...
  * encode the query as a vector
  * encode the code-document as a vector
  * compute the cosine similarity of the two vectors (where 1 is relevant, 0 is not-relevant)
  
We'll detail these next.

## Creating the encoders

How do we instantiate an encoder? There are two steps. First we need to figure out what is the _architecture_ of the model. This defines things like how many layers are in a neural network and how the layers are connected. In our case, _both_ of our encoders will use the RoBERTa architecture; as you might have guessed, RoBERTa is related to BERT and just has slightly different tweaks. There are [many BERT variants](https://towardsdatascience.com/exploring-bert-variants-albert-roberta-electra-642dfe51bc23) and for the purposes of this excise, you can safely think of RoBERTa as the same as BERT.

Second, once we have our model architecture, we need to specify which parameters we'll start with. You can think of the difference between the model architecture and parameters as if you were specifying a meal: The architecture is a bit like specifying the plates/bowls/container based on what kind of food you want and the parameters are like filling the container with a specific kind of food. There are many pre-trained sets of parameters for architectures, so a neural network starts with some existing knowledge of certain kinds of things (e.g., what human  language looks like,  what programming languages look like, or how to classify images). In neural network land, the [Huggingface Model Repository](https://huggingface.co/models) is a common place to look for parameters that people have shared with others.

Returning to our IR problem, in our setting (conveniently), both of our encoders will use the same architecture: `RobertaModel`. If you _really_ want to know more, the code for this is provided in `edited_roberta.py`, but many end-users of these models (like us) will never need to look at this kind of code--and you certainly don't to complete this homework!

## Writing the forward function 

The `forward` function defines how the neural network goes from inputs to outputs. In our case, we're going to feed the different inputs (query and document) to separate encoders and then compare the outputs. 


Your task here:
1. Define the two encoders(query encoder and code_encoder) which are RobertaModel loaded from edit_roberta.py. You should two arguments when creating the instance of the Roberta Model, which are config and add_pooling_layer(equal to False).
2. define and calculate the cosine similarity between two embeddings.
3. define the loss function and loss.

### TODOs in this block are worth 20 points total

In [3]:
class CodeSearchBiencoderModel(RobertaPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.config = config

        ## TODO: 
        # Fill in the following parts where you specify each encoder's architecture. 
        # You'll need to pass in "config" as an argument to the architecture's constructor
        # so it knows how to set things up.
        #
        # NOTE 1: Notice that we haven't specified the *parameters* here, just the architecture.
        # We'll fill in the parameters later
        #
        # NOTE 2: If you were ever curious how to do other kinds of non-text IR (e.g., images),
        # this part is where you'd specify a different kind of encoder architecture, such as
        # ResNet50 for encoding images. The rest of the code for this class would be mostly the same!
        # (The one caveat is that both models need to produce vector representations of the same size)
        
        self.query_encoder = RobertaModel(config)
        self.code_encoder = RobertaModel(config)

        # This is our loss function that determines how "good" our model's output is.
        # We'll use this in the forward() function to evaluate the model's outputs and
        # then return the predictions and loss.        
        self.loss_fn = BCEWithLogitsLoss()
        
        # This will initialize weights and apply final processing
        self.post_init()
 
    def forward(
        self,
        query_token_ids: Optional[torch.LongTensor] = None,
        code_token_ids: Optional[torch.LongTensor] = None,
        attention_mask: Optional[torch.FloatTensor] = None,
        labels: Optional[torch.LongTensor] = None,
       
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
       

        outputs = self.query_encoder(
            query_token_ids,
            attention_mask=attention_mask,
            
        )
        query_emb = outputs[0][:, 0, :]
        
        outputs_code = self.code_encoder(
            code_token_ids,
            attention_mask=attention_mask,
           
        )
        code_emb = outputs_code[0][:, 0, :]

        # TODO: using the cosine_similarity function (imported above),
        # compute the similarity of the query and code embeddings.
        cosine_sim = cosine_similarity(query_emb, code_emb)
                       
        # TODO: use the self.loss_fn (our loss function) to measure how good/bad
        # the predictions are. This function will return a value you should 
        # call "loss". You can see how to call the function here
        # https://pytorch.org/docs/stable/generated/torch.nn.BCELoss.html
        # 
        # NOTE: The "labels" input to this function is the ground truth
        # relevance scores (labels) for each input. You'll want to compare
        # the cosine similarities with these labels when calling the function
        #
        # NOTE 2: There are many kinds of loss functions so understanding how
        # to call them and which order the arguments go in is important
        loss = self.loss_fn(cosine_sim, labels)

        # Finally, let's return some output! Pytorch has provided
        # some structure for us to say what-is-what in the output
        # values. We'll return our loss (how "bad" the model's prediction was)
        # and the logits, which is our predictions. 
        #
        # You don't need to worry about the other two outputs.
        #
        # NOTE: Normally, we'd be passing the cosine similarity through
        # some non-linear function (like a sigmoid!) to get "logits" as
        # our output values. However, in a bi-encoder, often you just 
        # return the cosine similarity as the logits, so the name is wrong
        # but the value is what's expected. Later on, when we access the
        # "logits" part of the output, remember these are the query-doc
        # cosine similarity scores
        return SequenceClassifierOutput(
            loss=loss,
            logits=cosine_sim,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

# Configure the Models and Training Setting

Training deep learning models often involves _lots_ of hyperparameter decisions--not to mention a bunch of seemingly-random bookkeeping options for where and when to save things. We have defined these all for you (yay) but it's worth at least looking through to see what kinds of decisions you'll need to make. Most importantly, we've specified the number of epochs and the batch size (more on that later) so that the model trains quickly.

You will eventually need to edit the input file name to use the full dataset. Everything else can stay the same, though you're welcome to try changing some things and seeing what happens once you've completed the assignment.

In [4]:
class Args:
    def __init__(self):
        
        # Where to save things
        self.data_dir = './data'
        self.model_type = 'roberta'
        self.model_name_or_path = 'microsoft/codebert-base'
        self.task_name = 'codesearch'
        self.output_dir = './models'
        self.output_mode = 'codesearch'

        # These are going to be your most common hyperparameters to change.
        # If you want to do deep learning stuff, it's worth learning a bit
        # about what they are and what they do.
        self.train_batch_size = 64
        self.eval_batch_size = 64
        self.gradient_accumulation_steps = 1
        self.learning_rate = 1e-5
        self.weight_decay = 0.0
        self.adam_epsilon = 1e-8
        self.max_grad_norm = 1.0
        self.num_train_epochs = 3 # NOTE: Change this to 1 if debugging so it runs faster
        self.max_steps = -1
        self.warmup_steps = 0
        self.n_gpu = 1
        self.no_cuda = False

        # These are mostly configuration options for which pieces to run
        self.config_name = ""
        self.tokenizer_name = ""
        self.cache_dir = ""
        self.max_seq_length = 200
        self.do_train = True
        self.do_eval = True
        self.do_predict = False
        self.evaluate_during_training = False
        self.do_lower_case = False

        # How often we save things
        self.logging_steps = 1000
        self.save_steps = 1000
        self.eval_all_checkpoints = False
        self.overwrite_output_dir = True
        self.overwrite_cache = True
        self.seed = 42
        
        # Ignore all of these
        self.fp16 = False
        self.fp16_opt_level = 'O1'
        self.local_rank = -1
        self.server_ip = ""
        self.server_port = ""
        
        # Input and output files.
        #
        # TODO: Change the training file to train_300k.txt when ready
        #
        self.train_file = "train_300k.txt" # CHANGE ME WHEN READY TO TRAIN!!!!!
        self.dev_file = "valid.txt"
        self.test_file = "test_data.txt"
        self.pred_model_dir = './models/checkpoint-best'
        self.test_result_dir = './results/'

args = Args()

In the following blocks, we are starting to train our model. Here we will firstly define the train function.

In the train function, we will define the procedure of training the model, the main steps are:
1. Define the dataloader (to do). You should use the function DataLoader(). Three arguments are required for you to input, which are dataset, batch_size, sampler.
2. set the gradient to zero and train the model using back propagation.


# Define the training process

Let's see how the training procedure works! The code block below specifies how we'll train our model. There's one part for you to fill in that loads the data using the `DataLoader` class. The rest is helpful to understand how 

### TODO in this block is worth 10 points

In [5]:
def train(args, train_dataset, model, tokenizer, optimizer):
    """ Train the model """

    # The sampler specifies how we should access the training data, which
    # in this case is in a random order
    train_sampler = RandomSampler(train_dataset)
    
    # TODO: Initailize the DataLoader (https://pytorch.org/docs/stable/data.html)
    # so that it 
    # - loads from the provided train_dataset 
    # - samples using our sampler
    # - uses the specified batch size
    #
    # NOTE: The batch size is pretty important! This says how many examples to train on 
    # at one time. If you recall, we talked about Stocastic Gradient Descent (SGD) that
    # updates based on one instance at a time (e.g., changing the dog t-shirt size after seeing one dog)
    # versus Gradient Descent (GD) that updates after all the data. SGD is much faster to
    # converge to the "right" parameters but can make many missteps. The batch size 
    # says we can look at more than one instance at a time in determining how to update
    # our parameters (e.g., look at a few dogs at a time to determine how to best update 
    # the t-shirt size, rather than just one dog or all the dogs)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args.train_batch_size)
    
    # How many total steps we'll take
    t_total = len(train_dataloader) //  args.num_train_epochs

    # The scheduler helps decide how quickly to update the weights based on how much
    # training data we've seen. 
    scheduler = get_linear_schedule_with_warmup(optimizer, args.warmup_steps, t_total)
    checkpoint_last = os.path.join(args.output_dir, 'checkpoint-last')
    scheduler_last = os.path.join(checkpoint_last, 'scheduler.pt')
    if os.path.exists(scheduler_last):
        scheduler.load_state_dict(torch.load(scheduler_last))

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args.num_train_epochs)
    logger.info("  Total train batch size (w. parallel, distributed & accumulation) = %d",
                args.train_batch_size * args.gradient_accumulation_steps * (
                    torch.distributed.get_world_size() if args.local_rank != -1 else 1))
    logger.info("  Gradient Accumulation steps = %d", args.gradient_accumulation_steps)
    logger.info("  Total optimization steps = %d", t_total)

    global_step = args.start_step
    tr_loss, logging_loss = 0.0, 0.0
    best_acc = 0.0
    model.zero_grad()
    
    # Note that this "train_iterator" is just tdqm wrapper that prints out which
    # epoch we're currently in.     
    train_iterator = trange(args.start_epoch, int(args.num_train_epochs), desc="Epoch")
    
    set_seed(args) 
    
    # This tells pytorch that we're going to be changing the parameters so it needs
    # to start keeping track of stuff
    model.train()
    for idx, _ in enumerate(train_iterator):
        
        # Keep train of the training loss (how "bad" the performance is) for this epohch
        tr_loss = 0.0
        
        # For one epoch, loop over all the data, one batch at a time
        for step, batch in tqdm(enumerate(train_dataloader)):

            batch = tuple(t.to(args.device) for t in batch)
            inputs = {'query_token_ids': batch[0],
                      'code_token_ids': batch[1],
                      'labels': batch[3]}
            
            ouputs = model(**inputs)
            loss = ouputs[0]        
            
            # Do the back propagration to figure out which parameters to change.
            # It's that easy!
            loss.backward() 
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)

            tr_loss += loss.item()
            
            # Update the parameters of our model based on the gradient and whatever
            # else the optimizer is keeping track of
            optimizer.step() 
            scheduler.step()  
            
            # This sets the gradient to zero before doing next update so we don't
            # accidentally update the model based on the last batch's performance
            model.zero_grad() 
            global_step += 1

            if args.max_steps > 0 and global_step > args.max_steps:
                break

        # Once we finish an epoch, evaluate the model on the development data and see
        # how well it does. We'll use this information to decide which version of
        # the parameters to use.
        results = evaluate(args, model, tokenizer, checkpoint=str(args.start_epoch + idx))

        # 
        # Save the model and if we've already saved it, overwrite that saved model with 
        # the newly-trained parameters
        #
        last_output_dir = os.path.join(args.output_dir, 'checkpoint-last')
        if not os.path.exists(last_output_dir):
            os.makedirs(last_output_dir)
        model_to_save = model.module if hasattr(model,
                                                'module') else model 
        model_to_save.save_pretrained(last_output_dir)
        logger.info("Saving model checkpoint to %s", last_output_dir)
        idx_file = os.path.join(last_output_dir, 'idx_file.txt')
        with open(idx_file, 'w', encoding='utf-8') as idxf:
            idxf.write(str(args.start_epoch + idx) + '\n')

        torch.save(optimizer.state_dict(), os.path.join(last_output_dir, "optimizer.pt"))
        torch.save(scheduler.state_dict(), os.path.join(last_output_dir, "scheduler.pt"))
        logger.info("Saving optimizer and scheduler states to %s", last_output_dir)

        step_file = os.path.join(last_output_dir, 'step_file.txt')
        with open(step_file, 'w', encoding='utf-8') as stepf:
            stepf.write(str(global_step) + '\n')

        # Optional part 1 goes here

        #
        # If this model is better (on the training data) than the models from any of the 
        # past checkpoints, then keep a separate record of that too
        #
        if (results['acc'] > best_acc):
            best_acc = results['acc']
            output_dir = os.path.join(args.output_dir, 'checkpoint-best')
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            model_to_save = model.module if hasattr(model, 'module') else model  
            model_to_save.save_pretrained(output_dir)
            torch.save(args, os.path.join(output_dir, 'training_{}.bin'.format(idx)))
            logger.info("Saving model checkpoint to %s", output_dir)

            torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
            torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
            logger.info("Saving optimizer and scheduler states to %s", output_dir)

        if args.max_steps > 0 and global_step > args.max_steps:
            train_iterator.close()
            break

    return global_step, tr_loss / global_step

# Set up the training environment

This will get a few things ready for the model to train. You don't need to really do much in this block but it's worth seeing how it works if you want to train models in the future

In [6]:
# Setup CUDA so we can run on the GPU
device = torch.device("cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu")
args.device = device

# Setup logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO if args.local_rank in [-1, 0] else logging.WARN)

# Set seed
set_seed(args)

# This code will help us if we restart training and want to pick back up where we left off
args.start_epoch = 0
args.start_step = 0
checkpoint_last = os.path.join(args.output_dir, 'checkpoint-last')
if os.path.exists(checkpoint_last) and os.listdir(checkpoint_last):
    args.model_name_or_path = os.path.join(checkpoint_last, 'pytorch_model.bin')
    args.config_name = os.path.join(checkpoint_last, 'config.json')
    idx_file = os.path.join(checkpoint_last, 'idx_file.txt')
    with open(idx_file, encoding='utf-8') as idxf:
        args.start_epoch = int(idxf.readlines()[0].strip()) + 1

    step_file = os.path.join(checkpoint_last, 'step_file.txt')
    if os.path.exists(step_file):
        with open(step_file, encoding='utf-8') as stepf:
            args.start_step = int(stepf.readlines()[0].strip())
    logger.info("reload model from {}, resume from {} epoch".format(checkpoint_last, args.start_epoch))

12/06/2022 21:37:12 - INFO - __main__ -   reload model from ./models/checkpoint-last, resume from 3 epoch


# Task: Setting up the bi-encoder model and its parameters

### the TODO in this block is worth 10 points

In [7]:
# We'll specify some general configurations that tell the models what kind
# of parameters to use and how to turn incoming text/code data into identifiers for 
# processing with the neural network 
#
# We set num_labels = 1 because this is a regression class
# (compared to a classification task with many class labels)
num_labels = 1
config = RobertaConfig.from_pretrained('microsoft/codebert-base',
                                      num_labels=num_labels, finetuning_task=args.task_name)
# We'll treat relevance as a regression problem
config.problem_type = 'regression'

# If you remember from our neural language model part of the lecture, we talked
# about one language model that gets fed a series of words to predict the next
# and each word is mapped to an embedding. The "tokenizer" specifies how to 
# break up words into tokens but it frequently doesn't use just spaces!
# In fact, most tokenizers break words into *pieces* to reduce the size of the
# vocabularly (fewer embeddings!) so we need to specify which tokenizer to use.
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")


# Now comes the time to define our model. Let's specify the model class (which we'll need later)
model_class = CodeSearchBiencoderModel
# And we'll instantiate the model itself.
model = CodeSearchBiencoderModel(config)

# Now comes the magic where we specify the two encoders. Conveniently for us,
# there's actually a very recent langauge model that knows *both* code and human language!!
# We'll use this set of parameters to initialize *each* of our encoders. Over time,
# each encoder's parameters will start to become different since one side is going
# to learn how to encode queries better and the other will learn how to encode 
# code documents.
#
# NOTE: There's nothing stopping us from trying other parameters for the
# encoders too. If you're feeling curious you could swap in any RoBERTa model
# for the query encoder and it will just work.
#
# TODO: Initialize each of the coders using the "from_pretrained" method and
# specifying the pretrained model you want. Here, we'll use the CodeBERT model, 
# which is hosted on Huggingface https://huggingface.co/microsoft/codebert-base
# You should pass in the full name of the pretrained model (which includes the "/").
# Note that this code is going to look the same for both encoders and may 
# seem kind of easy to do but we want you to see how to do it yourself. :) 
model.query_encoder = RobertaModel.from_pretrained("microsoft/codebert-base")
model.code_encoder = RobertaModel.from_pretrained("microsoft/codebert-base")

# This will move the model's parameters onto the GPU so it runs fast
model.to(args.device)

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
     'weight_decay': args.weight_decay},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
# Remember how we talked about stochastic gradient descent (SGD)? Well, it's not
# the only way to update parameters. There are many (many) ways to do this
# and the usual standard is actually AdamW which uses a bit of bookkeeping to figure
# out how to update the weights more efficiently so the model learns faster.
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)

# If we're restarting, load the optimizer's state at the last time step
optimizer_last = os.path.join(checkpoint_last, 'optimizer.pt')
if os.path.exists(optimizer_last):
    optimizer.load_state_dict(torch.load(optimizer_last))

Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at microsoft/codebert-base were not used when initializing RobertaModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).


# Do the training!

Finally!! Let's train that model and save it to a file so we can evaluate it

In [8]:
logger.info("Training/evaluation parameters %s", args)

# Load in the training dataset. Here, we've handled most of the data preprocessing for you
train_dataset = load_and_cache_examples(args, args.task_name, tokenizer, ttype='train')

# Call the training function that we defined above
global_step, tr_loss = train(args, train_dataset, model, tokenizer, optimizer)
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()

# Create output directory if needed
if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

# Save the trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
logger.info("Saving model checkpoint to %s", args.output_dir)
model_to_save = model.module if hasattr(model, 'module') else model  
model_to_save.save_pretrained(args.output_dir)
tokenizer.save_pretrained(args.output_dir)

# Good practice: save your training arguments together with the trained model
torch.save(args, os.path.join(args.output_dir, 'training_args.bin'))

# Load a trained model and vocabulary that you have fine-tuned
model = AutoModel.from_pretrained(args.output_dir)
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)
model.to(args.device)

12/06/2022 21:37:28 - INFO - __main__ -   Training/evaluation parameters <__main__.Args object at 0x1502c5bbb2e0>
12/06/2022 21:37:28 - INFO - run_classifier -   Loading features from cached file ./data/cached_train_train_300k_pytorch_model.bin_200_codesearch
12/06/2022 21:37:28 - INFO - run_classifier -   Creating features from dataset file at ./data
12/06/2022 21:37:28 - INFO - utils -   LOOKING AT ./data/train_300k.txt
12/06/2022 21:37:30 - INFO - utils -   Writing example 0 of 247302
12/06/2022 21:37:30 - INFO - utils -   *** Example ***
12/06/2022 21:37:30 - INFO - utils -   guid: train-0
12/06/2022 21:37:30 - INFO - utils -   query_token_ids: 0 18377 10 762 8 10 14792 28700 25 5129 30 36410 7954 36418 5368 41 3901 4327 9 42766 479 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/06/2022 21:37:30 - INFO - utils -   code_token_ids: 0 9232 13075 7954 36 3741 29 2156 45655 2156 14792 2156 3000 2156 762 4839 4832 671 3741 29 36 45655 2156 3000 2156 762 2156 1009 1009 14792

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

# Evaluate the best model on the test data

Our code in training keeps track of how the model is doing and currently keeps around the files for the "best performing" model on the training data. How well does this model do on the test data? Let's find out!

In [9]:
# Evaluation
checkpoint = args.output_dir

logger.info("Evaluate the following checkpoint: %s", checkpoint)

print(checkpoint)
global_step = ""
model = model_class.from_pretrained(checkpoint)
model.to(args.device)
result = evaluate(args, model, tokenizer, checkpoint=checkpoint, prefix=global_step)
result = dict((k + '_{}'.format(global_step), v) for k, v in result.items())
print(result)

    
# Optional part 3 goes here


12/06/2022 21:40:54 - INFO - __main__ -   Evaluate the following checkpoint: ./models


./models


12/06/2022 21:40:56 - INFO - run_classifier -   Loading features from cached file ./data/cached_dev_valid_pytorch_model.bin_200_codesearch
12/06/2022 21:40:56 - INFO - run_classifier -   Creating features from dataset file at ./data
12/06/2022 21:40:56 - INFO - utils -   LOOKING AT ./data/valid.txt
12/06/2022 21:40:57 - INFO - utils -   Writing example 0 of 46213
12/06/2022 21:40:57 - INFO - utils -   *** Example ***
12/06/2022 21:40:57 - INFO - utils -   guid: dev-0
12/06/2022 21:40:57 - INFO - utils -   query_token_ids: 0 6407 13851 37357 5 2324 828 2622 30 5222 41 3169 19470 463 36 192 20387 39891 463 37380 1215 347 4839 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/06/2022 21:40:57 - INFO - utils -   code_token_ids: 0 9232 386 1215 9981 10845 36 1403 2156 2345 2156 1100 2156 425 5457 9291 2156 414 5457 9291 2156 17017 5457 9291 2156 923 5457 321 2156 1123 5457 883 612 4839 4832 18088 1403 479 18134 642 4345 1215 9981 10845 16 9291 2156 22 44307 554 48726 113 1403 479 181

{'acc_': 0.5000108194663839, 'f1_': 0.6666762839007502, 'acc_and_f1_': 0.583343551683567}


# Doing Inference on the Test Dataset

Finally, let's estimate the relevance scores for the query-document pairs in our test dataset. The test dataset **test_data.csv** contains each pair of 99 queries and 958 documents, which in total adds up to 94,842 query-document annotations (compare that with the project update number!). 

For ease of this exercise, we've already processed the data into a ready-to-go format in **test_data.txt** which is required by the model. In the following block, for each query-document pair, we generate a prediction score that measure the relevance of that pair by feeding the pair as inputs to the model's `forward` function (note that in pytorch if you have some model, doing `model(inputs)` and `model.forward(inputs)` is the same--it's trying to emphasize thinking of these as functions!). 

Once we have the model predictions, we'll create a new dataframe that contains (1) the query id, (2) the document id, and (3) the relevance score for that pair. We'll hand this dataframe off to Part 2 so you can finish up your GPU work.

You should adjust the directory path accordingly in order to successfully do the inference. Check what you got in the result.txt file and write this back into the **test_data.csv** that adds an additional column "sim". Later In Part 2, you will incorporate the prediction score into the learning to rank model to see if it can improve the performance of ranking,

### Implementation notes

This implementation works because we've aligned the `test_data.txt` and `test_data.csv` files so they're in the same order. That means you can write a long list of similarities and then add it back to the test data's DataFrame and it will Just Work™. However, in production settings, it's often useful to keep identifiers with the data as much as possible so that you don't just have a file of predictions and instead can write the predictions with the query/document identifiers (or whatever data you're working with).

In this homework's setup, we're precomputed the relevance scores for later integration with some overall ranking function (done in Part 2). To get these, we re-encode everything for each step. In commercial systems, what is typically done is the documents are encoded once and then cached, much like how our inverted index caches the terms in each document. Then when a new queries arrives, we only have to encode it and compare it with the cached document emeddings. This saves a lot of time! Thankfully our dataset is quite small here so we don't need to do that, but the ability to precompute and cache embeddings is worth remembering why bi-encoders are helpful and efficient for ranking.

In [10]:
# This tells the model that we're switching to evaluation mode (rather than training)
# so it should turn off any training-specific functionality that could make it slower
# or interfere with our results.
model.eval()
    
# Note here: we're loading the test set ***in sequential order***. This is critical
# for the next step because we need to map these predictions to query-document pairs.
# In training, we want to see a random order, but typically not during test.
eval_dataset, instances = load_and_cache_examples(args, "codesearch", tokenizer, ttype='test')
eval_sampler = SequentialSampler(eval_dataset)
eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args.eval_batch_size)

# This data structures will have our predictions and we'll fill them as we process each batch
relevance_predictions = np.array([])

for batch in tqdm(eval_dataloader, desc="Evaluating"):
    batch = tuple(t.to(args.device) for t in batch)

    # Get the model's cosine similarity for the query-document pairs
    # we pass as input. This no_grad() call also tells pytorch that
    # we're doing evaluation so pytorch doesn't have to keep track
    # of any gradients for updating the model (e.g., remember how
    # in the dog t-shirt fitting, we had to keep around how much to change
    # the t-shirt sizes).
    with torch.no_grad():
        
        # Prepare the inputs
        inputs = {'query_token_ids': batch[0],
                  'code_token_ids': batch[1],
                  'labels': batch[3]}
       
        # Note that this is a list of outputs, which includes the cosine
        # similarity, among other stuff
        outputs = model(**inputs)
        
    # Let's pull out just the cosine similarity
    _, cosine_sim = outputs[:2] 
    
    # Pytorch works with "tensors" which are just like fancy numpy arrays.
    # One main difference is that the tensor might "live" on a GPU, which 
    # means we need to copy it into regular computer memory to use it.
    # Here, we'll call .cpu() to get the value back off the GPU and then
    # convert the similarities to numpy. Remember, we're getting a list
    # of similarities back out!
    cosine_sims = cosine_sim.cpu().numpy()
    
    # Add these similarities to our current similarities
    relevance_predictions = np.append(relevance_predictions, cosine_sims, axis=0)

if not os.path.exists(args.test_result_dir):
    os.makedirs(args.test_result_dir)

output_test_file = os.path.join(args.test_result_dir, 'relevance-scores.csv')

with open(output_test_file, "w") as outf:
    logger.info("***** Writing relevance predictions *****")
    all_logits = relevance_predictions.tolist()
    
    # Note that we write these all as one big list. In the next step,
    # we'll merge these with the data frame
    outf.write(",".join([str(item) for item in all_logits]))
    
# Optional part 3 goes here

12/06/2022 21:44:04 - INFO - run_classifier -   Loading features from cached file ./data/cached_test_test_data_pytorch_model.bin_200_codesearch
12/06/2022 21:44:04 - INFO - run_classifier -   Creating features from dataset file at ./data
12/06/2022 21:44:04 - INFO - utils -   LOOKING AT ./data/test_data.txt
12/06/2022 21:44:04 - INFO - utils -   Writing example 0 of 94842
12/06/2022 21:44:04 - INFO - utils -   *** Example ***
12/06/2022 21:44:04 - INFO - utils -   guid: test-0
12/06/2022 21:44:04 - INFO - utils -   query_token_ids: 0 29631 740 36245 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
12/06/2022 21:44:04 - INFO - utils -   code_token_ids: 0 9232 3116 1215 49079 1640 49451 6 414 6 45382 8660 5214 6842 846 1215 495 3721 3755 2068 2076 3256 49434 21062 2274 414 7 47896 2870 4832 46669 48786 35 766 9 47896 2870 7 3116 414 7 4832 12528 48786 35 7031 4832 46669 414 35 2274 414 7 3116 7 47896 2870 4832 12528 414 35 295 35187 8932 4832 46

### TODO: Merge the predictions with the query/doc pairs (10 points)

The numpy array `relevance_predictions` now contains a list of all the similarities, which we'll need to merge with the test data. Conveniently, these predictions appear in the exact same order as the query-document pairs in the `data/test_data.csv` file.

Your task is to read in `test_data.csv` as a dataframe and merge these relevance predictions as a new column called "sim". We'll export this dataframe to a separate file with just a few columns for better efficiency. Write a _new_ file with a subset of this dataframe containing only the columns
* "sim"
* "qid" (the query id)
* "docno" (the document id)
These last two columns match the pyterrier naming conventions, which you'll need for Part 2.

In [11]:
df = pd.read_csv('data/test_data.csv')
df.head()

,Unnamed: 0,Query,GitHubUrl,code,Query_id,Doc_id,relevance,Language
0,0,write csv,https://github.com/sentinel-hub/sentinelhub-py...,"def write_csv(filename, data, delimiter=CSV_DE...",0,0,3,Python
1,1,write csv,https://github.com/jordanjoz1/flickr-views-cou...,"def write_to_csv(fname, header, rows): with op...",0,1,3,Python
2,2,write csv,https://github.com/fastai/fastai/blob/9fb84a5c...,def write_csv(self): # Get first element's fil...,0,2,3,Python
3,3,write csv,https://github.com/majerteam/sylk_parser/blob/...,"def to_csv(self, fbuf, quotechar='""', delimite...",0,3,3,Python
4,4,write csv,https://github.com/gem/oq-engine/blob/8294553a...,"def write_to_csv(self, filename): ''' Exports ...",0,4,3,Python


# _Optional TODO_: Evaluating the different models (20 points total; this is part 1)

 In the code above, we save the model's parameter for the most recent epoch and an extra directory for saving the model with the highest accuracy so far on the validation data. How much training does the model actually need to recognize relevance? Would one epoch be enough? What if we did 10? or 100? (100 might be too many for Great Lakes limits...). In this **optional part**, we'll describe a series of steps you can take to explore this part!
 
Most of this optional part consists of changing or extending the code above using regular python/pandas things (no deep learning) so this is accessible to anyone. It will require you to figure out how some of the code does work though, so it's useful in general. 
 
Here's what you need to do:
* Right now the model trains for 3 epochs total. Increase that number to 5 or more. There's a 3-hour limit per session for GPUs in Great Lakes so if you've completed all of part 1, it's worth getting a fresh session to get all 3 hours again. You can increase the number of epochs if you want too.
* When training, we save the last checkpoint (overwriting the previous result) and also see if this is the "best" model and save that too. You will need to add more code here to save the model after every epoch. The code to do the saving is already shown in that block, so you'll need to figure out which parts to re-use _and_ be sure to change the directory. Look for "Optional part 1 goes here" on where to start
* After training completes, let's see how well each of the models does on the test set. We've already provided some code that does the evaluation on the best performing model. Add more code so that it evaluates the models you just saved for each epoch (look for "Optional part 2 goes here") and make a plot of the performance on the test set
* To measure the impact on NDCG, we'll need to calculate the different bi-encoders' relevance estimates to different files to use in Part 2. You'll need to add more code that loads in each of these models from the checkpoint directories and runs the inference
* In the Part 4, just write the different models' predictions to separate files. You might combine this with the part 3 code if it's easier to do there. We haven't marked a spot for it explicitly, but you'll use these files in part 2.

The output of Part 1 should be a plot showing the F1 performance per epoch and a list of files for each epoch's trained model's relevance predictions.